In [33]:
import csv
import pandas as pd
import datetime as dt
from datetime import datetime
from itertools import islice
import numpy as np
from openpyxl import load_workbook
import collections
import openpyxl as op
import math
import xlsxwriter
from openpyxl import load_workbook

In [34]:
exportFile = 'Model construction CWM bgnbd2.xlsx'
excelBook = load_workbook(exportFile)
del excelBook["BGNBD Estimation"]
writer = pd.ExcelWriter(exportFile, engine = 'openpyxl')
writer.book = excelBook
writer.sheets = dict((ws.title, ws) for ws in excelBook.worksheets)

In [35]:
grgFile = "GRG Running.xlsx"
df_GRG = pd.read_excel(grgFile, header=None, nrows = 5, usecols = [0,1])
df_BGNBD = pd.read_excel(grgFile, skiprows=6)
df_BGNBD

,Constituent ID,Name,x (#donations),t_x (last gift),T (total time span),ln(.),ln(A_1),ln(A_2),ln(A_3),ln(A_4)
0,1927780,"Angeles, Joey D.",25,3.208219,3.794521,0,0,0,0,0
1,1927778,"Ayika, Christopher",0,0.000000,3.791781,0,0,0,0,0
2,1927784,"Boland, Patrick T.",6,0.172603,3.786301,0,0,0,0,0
3,1927785,"Callahan, Patrick J.",0,0.000000,3.775342,0,0,0,0,0
4,1927788,"Rowell, Esperanza",3,3.021918,3.767123,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
2021,1930534,"Chmielarczyk, Allison",0,0.000000,2.638356,0,0,0,0,0
2022,1929858,"Grove, Karen E.",3,2.350685,2.635616,0,0,0,0,0
2023,1929859,"Leipr, Michael H.",0,0.000000,2.635616,0,0,0,0,0
2024,1929860,"Foy, Charles B.",0,0.000000,2.635616,0,0,0,0,0


In [36]:
r = df_GRG.iloc[0,1]
alpha = df_GRG.iloc[1,1]
a = df_GRG.iloc[2,1]
b = df_GRG.iloc[3,1]

In [37]:
df_input = pd.read_csv("Input.csv")
inputList = df_input["Values"].tolist()
df_input

beginning_date = dt.datetime(int (inputList[1]), int (inputList[2]), int (inputList[3]))
predict_thru = dt.datetime(int (inputList[4]), int (inputList[5]), int (inputList[6]))

def datevalue(date1):
    temp = dt.datetime(1899, 12, 30)    # Note, not 31st Dec but 30th!
    delta = date1 - temp
    return float(delta.days) + (float(delta.seconds) / 86400)

def valuedate(fh):
    fh = int(fh)
    return datetime.fromordinal(datetime(1900, 1, 1).toordinal() + fh - 2)

beginning_date = datevalue(beginning_date)
predict_thru = datevalue(predict_thru)

endDate = beginning_date + .5*(predict_thru-beginning_date)
firstTimeTo = beginning_date + .31 * .5 * (predict_thru-beginning_date)
print(endDate)

41931.0


In [38]:
df_ext = pd.DataFrame()
tuntil = (predict_thru - beginning_date + 1)/365
print(tuntil)
n = 1/365
t = [n]
while n <= tuntil:
    n = n + (1/365)
    t.append(n)
df_ext['t'] = t
df_ext['E(X(t))'] = 0
df_ext['2F1'] = 0
df_ext

7.602739726027397


,t,E(X(t)),2F1
0,0.002740,0,0
1,0.005479,0,0
2,0.008219,0,0
3,0.010959,0,0
4,0.013699,0,0
...,...,...,...
2770,7.591781,0,0
2771,7.594521,0,0
2772,7.597260,0,0
2773,7.600000,0,0


In [39]:
z=[]
for index, rows in islice(df_ext.iterrows(), 0, None):
    z.append(df_ext.iloc[index,0] / (alpha + df_ext.iloc[index,0]))
df_ext['z']=z
df_ext['0'] = 1
df_ext

,t,E(X(t)),2F1,z,0
0,0.002740,0,0,0.002732,1
1,0.005479,0,0,0.005450,1
2,0.008219,0,0,0.008152,1
3,0.010959,0,0,0.010840,1
4,0.013699,0,0,0.013514,1
...,...,...,...,...,...
2770,7.591781,0,0,0.883610,1
2771,7.594521,0,0,0.883647,1
2772,7.597260,0,0,0.883684,1
2773,7.600000,0,0,0.883721,1


In [40]:
count = 1
ext_cols = []
extCond = True
while (extCond):
    for index, rows in islice(df_ext.iterrows(), 0, None):
        ext_cols.append(df_ext.iloc[index, count+3]*df_ext.iloc[index, 3]*(r+count-1)*(b+count-1)/((a+b-1+count-1)*count))
    if (ext_cols[-1] < 9e-11):
        extCond = False
    else:
        df_ext[str(count)] = ext_cols
        count = count + 1
        ext_cols = [];
df_ext

,t,E(X(t)),2F1,z,0,1,2,3,4,5,...,178,179,180,181,182,183,184,185,186,187
0,0.002740,0,0,0.002732,1,0.002732,0.000007,2.039655e-08,5.572828e-11,1.522631e-13,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,0.005479,0,0,0.005450,1,0.005450,0.000030,1.618422e-07,8.819739e-10,4.806397e-12,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2,0.008219,0,0,0.008152,1,0.008152,0.000066,5.417767e-07,4.416658e-09,3.600536e-11,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3,0.010959,0,0,0.010840,1,0.010840,0.000118,1.273799e-06,1.380812e-08,1.496815e-10,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
4,0.013699,0,0,0.013514,1,0.013514,0.000183,2.467771e-06,3.334826e-08,4.506521e-10,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2770,7.591781,0,0,0.883610,1,0.883610,0.780766,6.898925e-01,6.095957e-01,5.386447e-01,...,2.718717e-10,2.402285e-10,2.122682e-10,1.875622e-10,1.657318e-10,1.464422e-10,1.293978e-10,1.143371e-10,1.010294e-10,8.927055e-11
2771,7.594521,0,0,0.883647,1,0.883647,0.780832,6.899794e-01,6.096981e-01,5.387578e-01,...,2.739113e-10,2.420408e-10,2.138786e-10,1.889931e-10,1.670032e-10,1.475718e-10,1.304014e-10,1.152287e-10,1.018215e-10,8.997425e-11
2772,7.597260,0,0,0.883684,1,0.883684,0.780897,6.900663e-01,6.098004e-01,5.388708e-01,...,2.759647e-10,2.438656e-10,2.155001e-10,1.904340e-10,1.682834e-10,1.487093e-10,1.314120e-10,1.161267e-10,1.026193e-10,9.068302e-11
2773,7.600000,0,0,0.883721,1,0.883721,0.780963,6.901531e-01,6.099027e-01,5.389838e-01,...,2.780322e-10,2.457029e-10,2.171328e-10,1.918848e-10,1.695726e-10,1.498549e-10,1.324299e-10,1.170310e-10,1.034228e-10,9.139688e-11


In [41]:
df_save = df_ext
df_save

,t,E(X(t)),2F1,z,0,1,2,3,4,5,...,178,179,180,181,182,183,184,185,186,187
0,0.002740,0,0,0.002732,1,0.002732,0.000007,2.039655e-08,5.572828e-11,1.522631e-13,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,0.005479,0,0,0.005450,1,0.005450,0.000030,1.618422e-07,8.819739e-10,4.806397e-12,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2,0.008219,0,0,0.008152,1,0.008152,0.000066,5.417767e-07,4.416658e-09,3.600536e-11,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3,0.010959,0,0,0.010840,1,0.010840,0.000118,1.273799e-06,1.380812e-08,1.496815e-10,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
4,0.013699,0,0,0.013514,1,0.013514,0.000183,2.467771e-06,3.334826e-08,4.506521e-10,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2770,7.591781,0,0,0.883610,1,0.883610,0.780766,6.898925e-01,6.095957e-01,5.386447e-01,...,2.718717e-10,2.402285e-10,2.122682e-10,1.875622e-10,1.657318e-10,1.464422e-10,1.293978e-10,1.143371e-10,1.010294e-10,8.927055e-11
2771,7.594521,0,0,0.883647,1,0.883647,0.780832,6.899794e-01,6.096981e-01,5.387578e-01,...,2.739113e-10,2.420408e-10,2.138786e-10,1.889931e-10,1.670032e-10,1.475718e-10,1.304014e-10,1.152287e-10,1.018215e-10,8.997425e-11
2772,7.597260,0,0,0.883684,1,0.883684,0.780897,6.900663e-01,6.098004e-01,5.388708e-01,...,2.759647e-10,2.438656e-10,2.155001e-10,1.904340e-10,1.682834e-10,1.487093e-10,1.314120e-10,1.161267e-10,1.026193e-10,9.068302e-11
2773,7.600000,0,0,0.883721,1,0.883721,0.780963,6.901531e-01,6.099027e-01,5.389838e-01,...,2.780322e-10,2.457029e-10,2.171328e-10,1.918848e-10,1.695726e-10,1.498549e-10,1.324299e-10,1.170310e-10,1.034228e-10,9.139688e-11


In [42]:
df_ext = df_save
ext = []
df_ext['2F1'] = df_ext.iloc[:,4:-1].sum(axis=1)
extConst = (a+b-1)/(a-1)
for index, rows in islice(df_ext.iterrows(), 0, None):
    ext.append(extConst*(1-pow((alpha/(alpha+df_ext.iloc[index,0])),r) * df_ext.iloc[index,2]))
df_ext['E(X(t))'] = ext
df_ext

<ipython-input-42-d55cfb46e38a>:4: RuntimeWarning: divide by zero encountered in long_scalars
  extConst = (a+b-1)/(a-1)
<ipython-input-42-d55cfb46e38a>:6: RuntimeWarning: invalid value encountered in double_scalars
  ext.append(extConst*(1-pow((alpha/(alpha+df_ext.iloc[index,0])),r) * df_ext.iloc[index,2]))


,t,E(X(t)),2F1,z,0,1,2,3,4,5,...,178,179,180,181,182,183,184,185,186,187
0,0.002740,inf,1.002740,0.002732,1,0.002732,0.000007,2.039655e-08,5.572828e-11,1.522631e-13,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,0.005479,-inf,1.005479,0.005450,1,0.005450,0.000030,1.618422e-07,8.819739e-10,4.806397e-12,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2,0.008219,inf,1.008219,0.008152,1,0.008152,0.000066,5.417767e-07,4.416658e-09,3.600536e-11,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3,0.010959,-inf,1.010959,0.010840,1,0.010840,0.000118,1.273799e-06,1.380812e-08,1.496815e-10,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
4,0.013699,NaN,1.013699,0.013514,1,0.013514,0.000183,2.467771e-06,3.334826e-08,4.506521e-10,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2770,7.591781,inf,8.591781,0.883610,1,0.883610,0.780766,6.898925e-01,6.095957e-01,5.386447e-01,...,2.718717e-10,2.402285e-10,2.122682e-10,1.875622e-10,1.657318e-10,1.464422e-10,1.293978e-10,1.143371e-10,1.010294e-10,8.927055e-11
2771,7.594521,inf,8.594521,0.883647,1,0.883647,0.780832,6.899794e-01,6.096981e-01,5.387578e-01,...,2.739113e-10,2.420408e-10,2.138786e-10,1.889931e-10,1.670032e-10,1.475718e-10,1.304014e-10,1.152287e-10,1.018215e-10,8.997425e-11
2772,7.597260,inf,8.597260,0.883684,1,0.883684,0.780897,6.900663e-01,6.098004e-01,5.388708e-01,...,2.759647e-10,2.438656e-10,2.155001e-10,1.904340e-10,1.682834e-10,1.487093e-10,1.314120e-10,1.161267e-10,1.026193e-10,9.068302e-11
2773,7.600000,inf,8.600000,0.883721,1,0.883721,0.780963,6.901531e-01,6.099027e-01,5.389838e-01,...,2.780322e-10,2.457029e-10,2.171328e-10,1.918848e-10,1.695726e-10,1.498549e-10,1.324299e-10,1.170310e-10,1.034228e-10,9.139688e-11


In [43]:
calibLength = (endDate-beginning_date+1)/365
df_ns = pd.DataFrame()
df_ns["Constituent ID"] = df_BGNBD["Constituent ID"]
df_ns["T (total time span)"] = df_BGNBD["T (total time span)"]

In [44]:
firstPurch = []
T = df_ns['T (total time span)'].to_list()
for index, row in islice(df_ns.iterrows(), 0, None):
    firstPurch.append(calibLength - T[index])
df_ns['time of 1st purchase'] = (firstPurch)
maxTime = df_ns['time of 1st purchase'].max()
df_ns

,Constituent ID,T (total time span),time of 1st purchase
0,1927780,3.794521,0.008219
1,1927778,3.791781,0.010959
2,1927784,3.786301,0.016438
3,1927785,3.775342,0.027397
4,1927788,3.767123,0.035616
...,...,...,...
2021,1930534,2.638356,1.164384
2022,1929858,2.635616,1.167123
2023,1929859,2.635616,1.167123
2024,1929860,2.635616,1.167123


In [45]:
weekUntil = (predict_thru - beginning_date)/7
df_sls_top = pd.DataFrame()
trialDon = []
numDon = []
i = 1/365
while (i<=maxTime):
    trialDon.append(i)
    numDon.append(df_ns.loc[(df_ns['time of 1st purchase'] >= i-.001) & (df_ns['time of 1st purchase'] < i+.001)].count()[0])
    i = i + 1/365
df_sls_top["Time of Trial Donation"] = trialDon
df_sls_top["Number of Donors"] = numDon
df_sls_top

,Time of Trial Donation,Number of Donors
0,0.002740,0
1,0.005479,0
2,0.008219,1
3,0.010959,1
4,0.013699,0
...,...,...
420,1.153425,0
421,1.156164,3
422,1.158904,0
423,1.161644,1


In [46]:
df_sls = pd.DataFrame()
df_sls['t'] = t
df_sls['Cum. Rpt.'] = ""
df_sls['E(X(t))'] = ext
df_sls

,t,Cum. Rpt.,E(X(t))
0,0.002740,,inf
1,0.005479,,-inf
2,0.008219,,inf
3,0.010959,,-inf
4,0.013699,,NaN
...,...,...,...
2770,7.591781,,inf
2771,7.594521,,inf
2772,7.597260,,inf
2773,7.600000,,inf


In [47]:
slsCount = 0
slsCols = []
while (slsCount < df_sls_top.count()[0]):
    for index, rows in islice(df_sls.iterrows(), 0, None):
        if (df_sls.iloc[index, 0] <= df_sls_top["Time of Trial Donation"][slsCount]):
            slsCols.append(0)
        else:
            indx = 365*(df_sls.iloc[index, 0]-df_sls_top["Time of Trial Donation"][slsCount])
            slsCols.append(ext[round(indx)-1])
    df_sls[str(slsCount)] = slsCols
    slsCount = slsCount+1
    slsCols = []
df_sls

,t,Cum. Rpt.,E(X(t)),0,1,2,3,4,5,6,...,415,416,417,418,419,420,421,422,423,424
0,0.002740,,inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.005479,,-inf,inf,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.008219,,inf,-inf,inf,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.010959,,-inf,inf,-inf,inf,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.013699,,NaN,-inf,inf,-inf,inf,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2770,7.591781,,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
2771,7.594521,,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
2772,7.597260,,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
2773,7.600000,,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf


In [48]:
df_save1 = df_sls

In [49]:
# cumRpt = []
# cumColCnt = 0
# #for c in range(df_sls_top.count()[0]):
# print(df_sls.iloc[0, 3:-1])
# print(df_sls_top["Time of Trial Donation"])
# print(df_sls.iloc[0, 3:-1] * df_sls_top["Time of Trial Donation"])
# #for r in range (df_sls.count()[0]):
#     #print((df_sls.iloc[r, 3:-1] * df_sls_top["Time of Trial Donation"]))
#     #cumRpt.append(sum(df_sls.iloc[r, 3:-1] * df_sls_top["Time of Trial Donation"]))
# df_sls["Cum. Rpt"] = cumRpt
# df_sls
cumRpt = []
cumRptSum = 0
#print(df_sls_top["Number of Donors"])
#print(df_sls.iloc[1,3:-1].astype('float64'))
#print (df_sls_top["Number of Donors"].multiply((df_sls.iloc[0,3:-1].astype('float64')), level = 0))
for index, rows in islice(df_sls.iterrows(), 0, None):
    i = 0
    cumRptSum = 0
    #print(cumRptSum + df_sls.iloc[index, i+3])
    #print(numDon[i])
    while (i<df_sls_top.count()[0]): 
        cumRptSum = cumRptSum + df_sls.iloc[index, i+3] * numDon[i]
        i = i+1
    cumRpt.append(cumRptSum)
    #print(cumRptSum)
df_sls["Cum. Rpt."] = cumRpt
df_sls

<ipython-input-49-22e229d133e4>:23: RuntimeWarning: invalid value encountered in double_scalars
  cumRptSum = cumRptSum + df_sls.iloc[index, i+3] * numDon[i]


,t,Cum. Rpt.,E(X(t)),0,1,2,3,4,5,6,...,415,416,417,418,419,420,421,422,423,424
0,0.002740,0.0,inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.005479,NaN,-inf,inf,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.008219,NaN,inf,-inf,inf,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.010959,NaN,-inf,inf,-inf,inf,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.013699,NaN,NaN,-inf,inf,-inf,inf,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2770,7.591781,NaN,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
2771,7.594521,NaN,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
2772,7.597260,NaN,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
2773,7.600000,NaN,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf


In [50]:
df_checkCumRpt = pd.DataFrame()
week = list(range(1, int(weekUntil)+1))
df_checkCumRpt["week"] = week
df_checkCumRpt

,week
0,1
1,2
2,3
3,4
4,5
...,...
391,392
392,393
393,394
394,395


In [51]:
checkDon = [cumRpt[6]]
checkWeekDon = [checkDon[0]]
weekCount = 2
while (weekCount <= weekUntil):
    checkDon.append(cumRpt[7*weekCount-1])
    checkWeekDon.append(checkDon[weekCount-1] - checkDon[weekCount-2])
    weekCount = weekCount + 1
df_checkCumRpt["Cum. Rpt. Dons."] = checkDon
df_checkCumRpt["Weekly Rpt. Dons."] = checkWeekDon
df_checkCumRpt

,week,Cum. Rpt. Dons.,Weekly Rpt. Dons.
0,1,NaN,NaN
1,2,NaN,NaN
2,3,NaN,NaN
3,4,NaN,NaN
4,5,NaN,NaN
...,...,...,...
391,392,NaN,NaN
392,393,NaN,NaN
393,394,NaN,NaN
394,395,NaN,NaN


In [52]:
startDate = [beginning_date]
endDate = [startDate[0] + 6]
for index, row in islice(df_checkCumRpt.iterrows(), 1, None):
    startDate.append(startDate[index-1]+7)
    endDate.append(endDate[index-1]+7)
df_checkCumRpt['week start'] = (startDate)
df_checkCumRpt['end'] = (endDate)
df_checkCumRpt

,week,Cum. Rpt. Dons.,Weekly Rpt. Dons.,week start,end
0,1,NaN,NaN,40544.0,40550.0
1,2,NaN,NaN,40551.0,40557.0
2,3,NaN,NaN,40558.0,40564.0
3,4,NaN,NaN,40565.0,40571.0
4,5,NaN,NaN,40572.0,40578.0
...,...,...,...,...,...
391,392,NaN,NaN,43281.0,43287.0
392,393,NaN,NaN,43288.0,43294.0
393,394,NaN,NaN,43295.0,43301.0
394,395,NaN,NaN,43302.0,43308.0


In [53]:
t = [0]
t2 = [6/365]
for index, row in islice(df_checkCumRpt.iterrows(), 1, None):
    t.append(t[index-1]+7/365)
    t2.append(t2[index-1]+7/365)
df_checkCumRpt["num rpt don"] = ""
df_checkCumRpt["cumul."] = ""
df_checkCumRpt['t'] = (t)
df_checkCumRpt['t2'] = (t2)
df_checkCumRpt

,week,Cum. Rpt. Dons.,Weekly Rpt. Dons.,week start,end,num rpt don,cumul.,t,t2
0,1,NaN,NaN,40544.0,40550.0,,,0.000000,0.016438
1,2,NaN,NaN,40551.0,40557.0,,,0.019178,0.035616
2,3,NaN,NaN,40558.0,40564.0,,,0.038356,0.054795
3,4,NaN,NaN,40565.0,40571.0,,,0.057534,0.073973
4,5,NaN,NaN,40572.0,40578.0,,,0.076712,0.093151
...,...,...,...,...,...,...,...,...,...
391,392,NaN,NaN,43281.0,43287.0,,,7.498630,7.515068
392,393,NaN,NaN,43288.0,43294.0,,,7.517808,7.534247
393,394,NaN,NaN,43295.0,43301.0,,,7.536986,7.553425
394,395,NaN,NaN,43302.0,43308.0,,,7.556164,7.572603


In [54]:
dfraw2 = pd.read_excel(exportFile, sheet_name = "Raw data2")
dfraw2

,Constituent ID,Name,First Gift Date,Last Gift Date,Gift Date,Inactive?,adj. num gift,KEEP,rpt gift after cali
0,1927780,"Angeles, Joey D.",40547,43158,40547,False,1,0.0,NaN
1,1927780,"Angeles, Joey D.",40547,43158,40578,False,2,0.0,NaN
2,1927780,"Angeles, Joey D.",40547,43158,40606,False,3,0.0,NaN
3,1927780,"Angeles, Joey D.",40547,43158,40637,False,4,0.0,NaN
4,1927780,"Angeles, Joey D.",40547,43158,40667,False,5,0.0,NaN
...,...,...,...,...,...,...,...,...,...
5584,1929858,"Grove, Karen E.",40970,42031,41828,False,4,NaN,NaN
5585,1929858,"Grove, Karen E.",40970,42031,42031,False,0,0.0,NaN
5586,1929859,"Leipr, Michael H.",40970,40970,40970,True,1,NaN,NaN
5587,1929860,"Foy, Charles B.",40970,40970,40970,True,1,NaN,NaN


In [55]:
numRpt = []
cummul = []
weekStart = df_checkCumRpt['week start'].tolist()
weekEnd = df_checkCumRpt['end'].tolist()
t = df_checkCumRpt['t'].tolist()
t2 = df_checkCumRpt['t2'].tolist()
count99 = 0
countTotal99 = 0
for index, row in islice(df_checkCumRpt.iterrows(), 0, None):
    count99 = count99 + len(dfraw2.loc[(dfraw2['Gift Date']>=weekStart[index]) & (dfraw2['Gift Date']<=weekEnd[index])])
    count99 = count99 - len(df_ns.loc[df_ns['time of 1st purchase']>=(t[index]-.001)])
    count99 = count99 + len(df_ns.loc[df_ns['time of 1st purchase']>(t2[index]+.001)])
    countTotal99 = countTotal99+count99
    numRpt.append(count99)
    cummul.append(countTotal99)
    count99 = 0
df_checkCumRpt["num rpt don"] = numRpt
df_checkCumRpt["cumul."] = cummul
df_checkCumRpt

,week,Cum. Rpt. Dons.,Weekly Rpt. Dons.,week start,end,num rpt don,cumul.,t,t2
0,1,NaN,NaN,40544.0,40550.0,3,3,0.000000,0.016438
1,2,NaN,NaN,40551.0,40557.0,3,6,0.019178,0.035616
2,3,NaN,NaN,40558.0,40564.0,0,6,0.038356,0.054795
3,4,NaN,NaN,40565.0,40571.0,0,6,0.057534,0.073973
4,5,NaN,NaN,40572.0,40578.0,3,9,0.076712,0.093151
...,...,...,...,...,...,...,...,...,...
391,392,NaN,NaN,43281.0,43287.0,6,3539,7.498630,7.515068
392,393,NaN,NaN,43288.0,43294.0,7,3546,7.517808,7.534247
393,394,NaN,NaN,43295.0,43301.0,6,3552,7.536986,7.553425
394,395,NaN,NaN,43302.0,43308.0,0,3552,7.556164,7.572603


In [56]:
df_allCondExp = df_BGNBD.iloc[:,0:5].copy(deep=True)
endDate = beginning_date + .5*(predict_thru-beginning_date)
constT = (predict_thru - endDate)/365
df_allCondExp['t'] = constT
df_allCondExp["E(Y(t)|X=x,t_x,T)"] = ""
df_allCondExp["2F1"] = 0
df_allCondExp["a"] = df_allCondExp.iloc[:,2]+r
df_allCondExp["b"] = df_allCondExp.iloc[:,2] + b
df_allCondExp["c"] = df_allCondExp.iloc[:,2] + b + a - 1
df_allCondExp["z"] = constT/(alpha + df_allCondExp.iloc[:,4] + constT)
df_allCondExp['0'] = 1
df_allCondExp

,Constituent ID,Name,x (#donations),t_x (last gift),T (total time span),t,"E(Y(t)|X=x,t_x,T)",2F1,a,b,c,z,0
0,1927780,"Angeles, Joey D.",25,3.208219,3.794521,3.8,,0,26,26,26,0.442142,1
1,1927778,"Ayika, Christopher",0,0.000000,3.791781,3.8,,0,1,1,1,0.442283,1
2,1927784,"Boland, Patrick T.",6,0.172603,3.786301,3.8,,0,7,7,7,0.442565,1
3,1927785,"Callahan, Patrick J.",0,0.000000,3.775342,3.8,,0,1,1,1,0.443131,1
4,1927788,"Rowell, Esperanza",3,3.021918,3.767123,3.8,,0,4,4,4,0.443556,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021,1930534,"Chmielarczyk, Allison",0,0.000000,2.638356,3.8,,0,1,1,1,0.510866,1
2022,1929858,"Grove, Karen E.",3,2.350685,2.635616,3.8,,0,4,4,4,0.511054,1
2023,1929859,"Leipr, Michael H.",0,0.000000,2.635616,3.8,,0,1,1,1,0.511054,1
2024,1929860,"Foy, Charles B.",0,0.000000,2.635616,3.8,,0,1,1,1,0.511054,1


In [57]:
aceCount = 1
aceArr = []
while (aceCount <= 200):
    df_allCondExp[str(aceCount)]=df_allCondExp.iloc[:,aceCount+11]*(df_allCondExp.iloc[:,8]+aceCount-1)*(df_allCondExp.iloc[:,9]+aceCount-1)/((df_allCondExp.iloc[:,10]+aceCount-1)*aceCount)*df_allCondExp.iloc[:,11]
    aceCount = aceCount + 1
df_allCondExp

,Constituent ID,Name,x (#donations),t_x (last gift),T (total time span),t,"E(Y(t)|X=x,t_x,T)",2F1,a,b,...,191,192,193,194,195,196,197,198,199,200
0,1927780,"Angeles, Joey D.",25,3.208219,3.794521,3.8,,0,26,26,...,7.008880e-36,3.502427e-36,1.749163e-36,8.730408e-37,4.354964e-37,2.171114e-37,1.081761e-37,5.386827e-38,2.680957e-38,1.333535e-38
1,1927778,"Ayika, Christopher",0,0.000000,3.791781,3.8,,0,1,1,...,2.131954e-68,9.429272e-69,4.170408e-69,1.844501e-69,8.157919e-70,3.608110e-70,1.595806e-70,7.057983e-71,3.121627e-71,1.380643e-71
2,1927784,"Boland, Patrick T.",6,0.172603,3.786301,3.8,,0,7,7,...,1.810428e-57,8.262713e-58,3.770474e-58,1.720290e-58,7.847667e-59,3.579426e-59,1.632378e-59,7.443258e-60,3.393449e-60,1.546878e-60
3,1927785,"Callahan, Patrick J.",0,0.000000,3.775342,3.8,,0,1,1,...,3.073517e-68,1.361971e-68,6.035314e-69,2.674434e-69,1.185125e-69,5.251655e-70,2.327171e-70,1.031242e-70,4.569751e-71,2.024998e-71
4,1927788,"Rowell, Esperanza",3,3.021918,3.767123,3.8,,0,4,4,...,4.422703e-62,1.992369e-62,8.974641e-63,4.042315e-63,1.820578e-63,8.198887e-64,3.692047e-64,1.662443e-64,7.485030e-65,3.369831e-65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021,1930534,"Chmielarczyk, Allison",0,0.000000,2.638356,3.8,,0,1,1,...,1.934487e-56,9.882627e-57,5.048694e-57,2.579204e-57,1.317626e-57,6.731300e-58,3.438789e-58,1.756759e-58,8.974676e-59,4.584853e-59
2022,1929858,"Grove, Karen E.",3,2.350685,2.635616,3.8,,0,4,4,...,2.486755e-50,1.290723e-50,6.698821e-51,3.476398e-51,1.803959e-51,9.360312e-52,4.856470e-52,2.519522e-52,1.307023e-52,6.779783e-53
2023,1929859,"Leipr, Michael H.",0,0.000000,2.635616,3.8,,0,1,1,...,2.075506e-56,1.060695e-56,5.420723e-57,2.770281e-57,1.415763e-57,7.235309e-58,3.697632e-58,1.889689e-58,9.657327e-59,4.935414e-59
2024,1929860,"Foy, Charles B.",0,0.000000,2.635616,3.8,,0,1,1,...,2.075506e-56,1.060695e-56,5.420723e-57,2.770281e-57,1.415763e-57,7.235309e-58,3.697632e-58,1.889689e-58,9.657327e-59,4.935414e-59


In [58]:
aceCount1 = 0
while (aceCount1 <= 200):
    df_allCondExp["2F1"] = df_allCondExp["2F1"]+df_allCondExp.iloc[:,aceCount1+12]
    aceCount1 = aceCount1 + 1
df_allCondExp

,Constituent ID,Name,x (#donations),t_x (last gift),T (total time span),t,"E(Y(t)|X=x,t_x,T)",2F1,a,b,...,191,192,193,194,195,196,197,198,199,200
0,1927780,"Angeles, Joey D.",25,3.208219,3.794521,3.8,,3.893931e+06,26,26,...,7.008880e-36,3.502427e-36,1.749163e-36,8.730408e-37,4.354964e-37,2.171114e-37,1.081761e-37,5.386827e-38,2.680957e-38,1.333535e-38
1,1927778,"Ayika, Christopher",0,0.000000,3.791781,3.8,,1.793025e+00,1,1,...,2.131954e-68,9.429272e-69,4.170408e-69,1.844501e-69,8.157919e-70,3.608110e-70,1.595806e-70,7.057983e-71,3.121627e-71,1.380643e-71
2,1927784,"Boland, Patrick T.",6,0.172603,3.786301,3.8,,5.979194e+01,7,7,...,1.810428e-57,8.262713e-58,3.770474e-58,1.720290e-58,7.847667e-59,3.579426e-59,1.632378e-59,7.443258e-60,3.393449e-60,1.546878e-60
3,1927785,"Callahan, Patrick J.",0,0.000000,3.775342,3.8,,1.795754e+00,1,1,...,3.073517e-68,1.361971e-68,6.035314e-69,2.674434e-69,1.185125e-69,5.251655e-70,2.327171e-70,1.031242e-70,4.569751e-71,2.024998e-71
4,1927788,"Rowell, Esperanza",3,3.021918,3.767123,3.8,,1.043073e+01,4,4,...,4.422703e-62,1.992369e-62,8.974641e-63,4.042315e-63,1.820578e-63,8.198887e-64,3.692047e-64,1.662443e-64,7.485030e-65,3.369831e-65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021,1930534,"Chmielarczyk, Allison",0,0.000000,2.638356,3.8,,2.044428e+00,1,1,...,1.934487e-56,9.882627e-57,5.048694e-57,2.579204e-57,1.317626e-57,6.731300e-58,3.438789e-58,1.756759e-58,8.974676e-59,4.584853e-59
2022,1929858,"Grove, Karen E.",3,2.350685,2.635616,3.8,,1.749668e+01,4,4,...,2.486755e-50,1.290723e-50,6.698821e-51,3.476398e-51,1.803959e-51,9.360312e-52,4.856470e-52,2.519522e-52,1.307023e-52,6.779783e-53
2023,1929859,"Leipr, Michael H.",0,0.000000,2.635616,3.8,,2.045215e+00,1,1,...,2.075506e-56,1.060695e-56,5.420723e-57,2.770281e-57,1.415763e-57,7.235309e-58,3.697632e-58,1.889689e-58,9.657327e-59,4.935414e-59
2024,1929860,"Foy, Charles B.",0,0.000000,2.635616,3.8,,2.045215e+00,1,1,...,2.075506e-56,1.060695e-56,5.420723e-57,2.770281e-57,1.415763e-57,7.235309e-58,3.697632e-58,1.889689e-58,9.657327e-59,4.935414e-59


In [59]:
longNameArr = []
#((a+b+df_allCondExp.iloc[:,2]-1)/(a-1)*(1-pow(((alpha+df_allCondExp.iloc[:,4])/(alpha+df_allCondExp.iloc[:,4]+df_allCondExp.iloc[:,5])),(r+df_allCondExp.iloc[:,2]))*df_allCondExp.iloc[:,7])/(1+(df_allCondExp.iloc[:,2]>0)*a/(b+df_allCondExp.iloc[:,2]-1)*pow(((alpha+df_allCondExp.iloc[:,4])/(alpha+df_allCondExp.iloc[:,3])),(r+df_allCondExp.iloc[:,2]))))
for index, row in islice(df_allCondExp.iterrows(), 0, None):
    longNameArr.append((a+b+df_allCondExp.iloc[index,2]-1)/(a-1)*(1-pow(((alpha+df_allCondExp.iloc[index,4])/(alpha+df_allCondExp.iloc[index,4]+df_allCondExp.iloc[index,5])),(r+df_allCondExp.iloc[index,2]))*df_allCondExp.iloc[index,7])/(1+(df_allCondExp.iloc[index,2]>0)*a/(b+df_allCondExp.iloc[index,2]-1)*pow(((alpha+df_allCondExp.iloc[index,4])/(alpha+df_allCondExp.iloc[index,3])),(r+df_allCondExp.iloc[index,2]))))
df_allCondExp["E(Y(t)|X=x,t_x,T)"] = longNameArr
df_allCondExp

<ipython-input-59-9d21aaa3111e>:4: RuntimeWarning: divide by zero encountered in long_scalars
  longNameArr.append((a+b+df_allCondExp.iloc[index,2]-1)/(a-1)*(1-pow(((alpha+df_allCondExp.iloc[index,4])/(alpha+df_allCondExp.iloc[index,4]+df_allCondExp.iloc[index,5])),(r+df_allCondExp.iloc[index,2]))*df_allCondExp.iloc[index,7])/(1+(df_allCondExp.iloc[index,2]>0)*a/(b+df_allCondExp.iloc[index,2]-1)*pow(((alpha+df_allCondExp.iloc[index,4])/(alpha+df_allCondExp.iloc[index,3])),(r+df_allCondExp.iloc[index,2]))))
<ipython-input-59-9d21aaa3111e>:4: RuntimeWarning: invalid value encountered in long_scalars
  longNameArr.append((a+b+df_allCondExp.iloc[index,2]-1)/(a-1)*(1-pow(((alpha+df_allCondExp.iloc[index,4])/(alpha+df_allCondExp.iloc[index,4]+df_allCondExp.iloc[index,5])),(r+df_allCondExp.iloc[index,2]))*df_allCondExp.iloc[index,7])/(1+(df_allCondExp.iloc[index,2]>0)*a/(b+df_allCondExp.iloc[index,2]-1)*pow(((alpha+df_allCondExp.iloc[index,4])/(alpha+df_allCondExp.iloc[index,3])),(r+df_allCon

,Constituent ID,Name,x (#donations),t_x (last gift),T (total time span),t,"E(Y(t)|X=x,t_x,T)",2F1,a,b,...,191,192,193,194,195,196,197,198,199,200
0,1927780,"Angeles, Joey D.",25,3.208219,3.794521,3.8,-inf,3.893931e+06,26,26,...,7.008880e-36,3.502427e-36,1.749163e-36,8.730408e-37,4.354964e-37,2.171114e-37,1.081761e-37,5.386827e-38,2.680957e-38,1.333535e-38
1,1927778,"Ayika, Christopher",0,0.000000,3.791781,3.8,NaN,1.793025e+00,1,1,...,2.131954e-68,9.429272e-69,4.170408e-69,1.844501e-69,8.157919e-70,3.608110e-70,1.595806e-70,7.057983e-71,3.121627e-71,1.380643e-71
2,1927784,"Boland, Patrick T.",6,0.172603,3.786301,3.8,NaN,5.979194e+01,7,7,...,1.810428e-57,8.262713e-58,3.770474e-58,1.720290e-58,7.847667e-59,3.579426e-59,1.632378e-59,7.443258e-60,3.393449e-60,1.546878e-60
3,1927785,"Callahan, Patrick J.",0,0.000000,3.775342,3.8,NaN,1.795754e+00,1,1,...,3.073517e-68,1.361971e-68,6.035314e-69,2.674434e-69,1.185125e-69,5.251655e-70,2.327171e-70,1.031242e-70,4.569751e-71,2.024998e-71
4,1927788,"Rowell, Esperanza",3,3.021918,3.767123,3.8,inf,1.043073e+01,4,4,...,4.422703e-62,1.992369e-62,8.974641e-63,4.042315e-63,1.820578e-63,8.198887e-64,3.692047e-64,1.662443e-64,7.485030e-65,3.369831e-65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021,1930534,"Chmielarczyk, Allison",0,0.000000,2.638356,3.8,NaN,2.044428e+00,1,1,...,1.934487e-56,9.882627e-57,5.048694e-57,2.579204e-57,1.317626e-57,6.731300e-58,3.438789e-58,1.756759e-58,8.974676e-59,4.584853e-59
2022,1929858,"Grove, Karen E.",3,2.350685,2.635616,3.8,inf,1.749668e+01,4,4,...,2.486755e-50,1.290723e-50,6.698821e-51,3.476398e-51,1.803959e-51,9.360312e-52,4.856470e-52,2.519522e-52,1.307023e-52,6.779783e-53
2023,1929859,"Leipr, Michael H.",0,0.000000,2.635616,3.8,NaN,2.045215e+00,1,1,...,2.075506e-56,1.060695e-56,5.420723e-57,2.770281e-57,1.415763e-57,7.235309e-58,3.697632e-58,1.889689e-58,9.657327e-59,4.935414e-59
2024,1929860,"Foy, Charles B.",0,0.000000,2.635616,3.8,NaN,2.045215e+00,1,1,...,2.075506e-56,1.060695e-56,5.420723e-57,2.770281e-57,1.415763e-57,7.235309e-58,3.697632e-58,1.889689e-58,9.657327e-59,4.935414e-59


In [60]:
df_pAlive = df_BGNBD.iloc[:,0:5].copy(deep=True)
paliveinfo = []
for index, row in islice(df_allCondExp.iterrows(), 0, None):
    paliveinfo.append(1/(1+(df_pAlive.iloc[index,2]>0)*(a/(b+df_pAlive.iloc[index,2]-1))*(pow(((alpha+df_pAlive.iloc[index,4])/(alpha+df_pAlive.iloc[index,3])),(r+df_pAlive.iloc[index,2])))))
df_pAlive["P(Alive) Info"] = paliveinfo
df_pAlive

<ipython-input-60-074ef2db0b12>:4: RuntimeWarning: divide by zero encountered in long_scalars
  paliveinfo.append(1/(1+(df_pAlive.iloc[index,2]>0)*(a/(b+df_pAlive.iloc[index,2]-1))*(pow(((alpha+df_pAlive.iloc[index,4])/(alpha+df_pAlive.iloc[index,3])),(r+df_pAlive.iloc[index,2])))))
<ipython-input-60-074ef2db0b12>:4: RuntimeWarning: invalid value encountered in multiply
  paliveinfo.append(1/(1+(df_pAlive.iloc[index,2]>0)*(a/(b+df_pAlive.iloc[index,2]-1))*(pow(((alpha+df_pAlive.iloc[index,4])/(alpha+df_pAlive.iloc[index,3])),(r+df_pAlive.iloc[index,2])))))


,Constituent ID,Name,x (#donations),t_x (last gift),T (total time span),P(Alive) Info
0,1927780,"Angeles, Joey D.",25,3.208219,3.794521,0.457003
1,1927778,"Ayika, Christopher",0,0.000000,3.791781,NaN
2,1927784,"Boland, Patrick T.",6,0.172603,3.786301,0.000318
3,1927785,"Callahan, Patrick J.",0,0.000000,3.775342,NaN
4,1927788,"Rowell, Esperanza",3,3.021918,3.767123,0.603166
...,...,...,...,...,...,...
2021,1930534,"Chmielarczyk, Allison",0,0.000000,2.638356,NaN
2022,1929858,"Grove, Karen E.",3,2.350685,2.635616,0.683987
2023,1929859,"Leipr, Michael H.",0,0.000000,2.635616,NaN
2024,1929860,"Foy, Charles B.",0,0.000000,2.635616,NaN


In [61]:
dfdp2 = pd.read_excel(exportFile, sheet_name = "Data Prep2")
dfdp2

,Constituent ID,Name,First Gift Date,Last Gift Date,Gift Date,Inactive?,adj. num gift,KEEP
0,1927780,"Angeles, Joey D.",40547,43158,41718,False,26,NaN
1,1927778,"Ayika, Christopher",40548,40548,40548,True,1,NaN
2,1927784,"Boland, Patrick T.",40550,40613,40613,True,7,NaN
3,1927785,"Callahan, Patrick J.",40554,40554,40554,True,1,NaN
4,1927788,"Rowell, Esperanza",40557,42403,41660,False,4,NaN
...,...,...,...,...,...,...,...,...
2021,1930534,"Chmielarczyk, Allison",40969,40969,40969,True,1,NaN
2022,1929858,"Grove, Karen E.",40970,42031,41828,False,4,NaN
2023,1929859,"Leipr, Michael H.",40970,40970,40970,True,1,NaN
2024,1929860,"Foy, Charles B.",40970,40970,40970,True,1,NaN


In [62]:
actNonlap = []
df_pAlive["Active"] = ~dfdp2["Inactive?"]*1
for index, row in islice(df_pAlive.iterrows(), 0, None):
    if df_pAlive.iloc[index,6] == 0:
        actNonlap.append(0)
    else:
        if (df_pAlive.iloc[index,4]-df_pAlive.iloc[index,3])>2:
            actNonlap.append(0)
        else:
            actNonlap.append(1)
df_pAlive["Act/Nonlap"] = actNonlap
df_pAlive

,Constituent ID,Name,x (#donations),t_x (last gift),T (total time span),P(Alive) Info,Active,Act/Nonlap
0,1927780,"Angeles, Joey D.",25,3.208219,3.794521,0.457003,1,1
1,1927778,"Ayika, Christopher",0,0.000000,3.791781,NaN,0,0
2,1927784,"Boland, Patrick T.",6,0.172603,3.786301,0.000318,0,0
3,1927785,"Callahan, Patrick J.",0,0.000000,3.775342,NaN,0,0
4,1927788,"Rowell, Esperanza",3,3.021918,3.767123,0.603166,1,1
...,...,...,...,...,...,...,...,...
2021,1930534,"Chmielarczyk, Allison",0,0.000000,2.638356,NaN,0,0
2022,1929858,"Grove, Karen E.",3,2.350685,2.635616,0.683987,1,1
2023,1929859,"Leipr, Michael H.",0,0.000000,2.635616,NaN,0,0
2024,1929860,"Foy, Charles B.",0,0.000000,2.635616,NaN,0,0


In [63]:
df_onlyx = df_pAlive.copy(deep=True)
df_onlyx

,Constituent ID,Name,x (#donations),t_x (last gift),T (total time span),P(Alive) Info,Active,Act/Nonlap
0,1927780,"Angeles, Joey D.",25,3.208219,3.794521,0.457003,1,1
1,1927778,"Ayika, Christopher",0,0.000000,3.791781,NaN,0,0
2,1927784,"Boland, Patrick T.",6,0.172603,3.786301,0.000318,0,0
3,1927785,"Callahan, Patrick J.",0,0.000000,3.775342,NaN,0,0
4,1927788,"Rowell, Esperanza",3,3.021918,3.767123,0.603166,1,1
...,...,...,...,...,...,...,...,...
2021,1930534,"Chmielarczyk, Allison",0,0.000000,2.638356,NaN,0,0
2022,1929858,"Grove, Karen E.",3,2.350685,2.635616,0.683987,1,1
2023,1929859,"Leipr, Michael H.",0,0.000000,2.635616,NaN,0,0
2024,1929860,"Foy, Charles B.",0,0.000000,2.635616,NaN,0,0


In [64]:
dfGRG.to_excel(writer, "BGNBD Estimation", index = False, header = False, startrow=0, startcol=0)
df_BGNBD.to_excel(writer, "BGNBD Estimation", index = False, startrow = 6, startcol=0)
df_ext.to_excel(writer, "E(X(t))", index = False)
df_ns.to_excel(writer, "n_s", index=False)
df_sls.to_excel(writer, "CumRptSls", index=False)
df_checkCumRpt.to_excel(writer, "Check CumRpt", index=False)
df_allCondExp.to_excel(writer, "All Cond. Exp.", index=False)
df_pAlive.to_excel(writer, "P(alive)", index=False)
df_onlyx.to_excel(writer, "only x>0", index=False)
writer.save()

NameError: name 'dfGRG' is not defined

In [ ]:
f= open("ModelCon.txt","w+")
f.write("Model Construction Successfully Completed")
f.close()